In [28]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [29]:
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
import pandas
from llama_parse import LlamaParse

In [31]:
# embed model

embed_model = HuggingFaceEmbeddings(
    model_name = "jhgan/ko-sroberta-nli",
    model_kwargs = {"device':'cpu"},
    encode_kwargs = {"normalize_embeddings":True}
)

c:\Users\jw\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:

llm = ChatOpenAI(
    temperature = 0,
    model_name = "gpt-4o-mini",
    max_tokens=500
)

In [ ]:
# Load Split Embed function

def LSE(fname):
    
    # Load
    loader = TextLoader(fname.page_content, encoding = "utf-8")
    data = loader.load()
    splitter = CharacterTextSplitter(
    separator = "//<>",
    # chunk_size = size,
    # chunk_overlap = overlap,
    length_function = len
    )

    # Split
    documents = splitter.split_documents(data)
    
    #Embed
    vectorstore = FAISS.from_documents(documents,
                                   embedding = embed_model,
                                   distance_strategy = DistanceStrategy.COSINE)
    
    return vectorstore

In [ ]:
#Prompt Chain Response function

def PCR(template, docs, query):
    # Prompt
    prompt = ChatPromptTemplate.from_template(template)

    # 이 부분은 뭔지 모르겠음
    # def format_docs(docs):
    #     return '\n\n'.join([d.page_content for d in docs])

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Run
    response = chain.invoke({'context': docs, 'question':query})
    
    return response

In [ ]:
# 공학인증 chain

# Engineering Chain
def EChain(docs, query):
    vectorstore1 = LSE(docs[0].page_content)
    response1 = PCR()

In [30]:
# load and split file_list

f_loader = TextLoader("file_list.txt", encoding = "utf-8")
f_data = f_loader.load()
f_splitter = CharacterTextSplitter(
    separator = ",",
    chunk_size = 20,
    chunk_overlap = 0,
    length_function = len
)

f_documents = f_splitter.split_documents(f_data)

Created a chunk of size 25, which is longer than the specified 20
Created a chunk of size 25, which is longer than the specified 20


3


In [ ]:
# embed and save file_list and retrieve

f_vectorstore = FAISS.from_documents(f_documents,
                                   embedding = embed_model,
                                   distance_strategy = DistanceStrategy.COSINE)


query = "2021학년도 소프트웨어학부 신입학자가 공학인증을 받고 졸업하기 위한 조건을 알려줘."

f_retriever = f_vectorstore.as_retriever(search_kwargs={'k': 2})

f_docs = f_retriever.get_relevant_documents(query)

page_content='공학인증제도 안내 - 졸업이수 요건.md' metadata={'source': 'file_list.txt'}


In [ ]:
if f_docs[0].page_content[0:5] == "공학인증제도":
    

In [33]:
# load and text split

loader = TextLoader(f_docs[0].page_content, encoding = "utf-8")
data = loader.load()
splitter = CharacterTextSplitter(
    separator = "//<>",
    # chunk_size = 4500,
    # chunk_overlap = 500,
    # length_function = len
)

documents = splitter.split_documents(data)

4

In [34]:
# save embeddings in vectorstore

vectorstore = FAISS.from_documents(documents,
                                   embedding = embed_model,
                                   distance_strategy = DistanceStrategy.COSINE)

In [36]:
# query and retirever

#query = '2019학년도 소프트웨어학부 신입학자가 졸업하기 위한 학점을 알려줘.'

retriever = vectorstore.as_retriever(search_kwargs={'k': 1})

docs = retriever.get_relevant_documents(query)

# for i in docs:
#     print(i.page_content, "\n")

In [37]:
# generation


# 당신은 광운대학교 학생인 사용자에게 사용자가 입력한 자신의 입학 연도를 기반으로 졸업 요건을 알려주는 인공지능 챗봇입니다.
#             졸업 이수학점 표는 총 7열입니다. 이때 7열중 교양은 (필수+균형)과 기초라는 2열로 다시 나누어집니다.
#             주전공학점(필수 포함)은 단일 전공시와 다전공 이수시로 2열로 나누어집니다.
#             표에서 이웃한 여러 셀이 동일한 값을 가질 경우 그 열의 같은 값을 가지는 이웃한 셀들을 통합하기도 합니다.
#             또한 이 문서는 각 졸업 요건이 동일한 입학 연도 단위 앞 부분에 가., 나., 다. ... 순으로 번호를 매깁니다.
#             이 중 어떤 부분을 참고해서 답변 하였는지 맨 처음에 명시하세요.


template = '''
            오직 한가지 한 단위만 참고해서 답변해야 하며, 당신이 참고했다 말하는 부분만을 기반으로 답변해야 합니다.
            생략되는 정보가 없어야 합니다.
            만약 표에 나타나 있는 데이터를 참고해야 한다면, 마크다운 언어를 이용해 표 형식으로 출력하되, 병합된 셀이 있는지를 감안하세요.
            오직 다음의 context에 기반하여 대답하세요. {context}, Question: {question}
            '''

prompt = ChatPromptTemplate.from_template(template)

# 이 부분은 뭔지 모르겠음
# def format_docs(docs):
#     return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = prompt | llm | StrOutputParser()

# Run
response = chain.invoke({'context': docs, 'question':query})
print(response)

2021학년도 소프트웨어학부 신입학자가 공학인증을 받고 졸업하기 위한 조건은 다음과 같습니다:

1. **최소 졸업 이수학점**: 133학점
2. **전공학점(필수 포함)**: 60학점 (설계 12학점 포함)
3. **교양 및 MSC(수학, 기초과학, 전산학)**: 12~30학점
   - 소프트웨어학부의 경우: 12학점
4. **교양 교과목 이수체계**: 해당 학과의 이수체계에 따라 이수
5. **졸업논문 및 소속 학과별 졸업요건**: 학과 홈페이지 참조

각 학과의 졸업요건 세부 사항은 해당 학과 내규에 따르므로 반드시 소속 학과로 문의해야 합니다.
